In [1]:
import random
import numpy as np

metodini "utilities"

In [11]:
def get_drone_customers(drone_tours):
    drone_customers = []
    for tour in drone_tours:
        for customer in tour:
            drone_customers.append(customer)

    return drone_customers

In [159]:
def get_truck_customers(truck_tours):
    truck_customers = []
    for tour in truck_tours:
        for customer in tour:
            truck_customers.append(customer)

    return truck_customers

In [15]:
def sort_by_angular(coordinates, c_drones, grad_seed):

    rad_seed = np.deg2rad(grad_seed)
    ref_point = coordinates[0]

    def polar_angle(coord):
        # Compute the angle in radians with respect to the reference point
        dx = coord[0] - ref_point[0]
        dy = coord[1] - ref_point[1]
        angle = np.arctan2(dy, dx)
        return angle

    def angular_distance(angle1, angle2):
        # Compute the absolute angular distance between two angles
        return min(abs(angle1 - angle2), 2 * np.pi - abs(angle1 - angle2))

    # Calculate the polar angles for all nodes in c_drones
    angles = [polar_angle(coordinates[i]) for i in c_drones]

    # Calculate the angular distance from grad_seed
    distances = [angular_distance(angle, rad_seed) for angle in angles]

    # Pair the indices with their angular distances
    indexed_distances = list(zip(c_drones, distances))

    # Sort based on angular distance
    indexed_distances.sort(key=lambda x: x[1])

    # Extract the sorted indices
    sorted_c_drones = [index for index, _ in indexed_distances]

    return sorted_c_drones

#test ok

In [57]:
def sort_by_euclidean_distance(distance_matrix, c_seed):
    distances = [distance_matrix[c_seed][i] for i in range(1, len(distance_matrix[c_seed]))]

    sorted_c = np.argsort(distances)
    sorted_c = [x + 1 for x in sorted_c]

    return sorted_c

# test ok

In [24]:
def remove_drone_customer(drone_tour, absent_customers, customer_to_remove):
    new_tour = drone_tour
    new_tour.remove(customer_to_remove)
    new_absent_vector = absent_customers + [customer_to_remove]

    return new_tour, new_absent_vector

#test ok

In [36]:
def max_string_length(tours, N, L_max):
    average_tour_cardinality = (N -1) / (len(tours[0]) + len(tours[1]))
    return min(average_tour_cardinality, L_max)

#test ok

In [38]:
def n_strings_to_remove(c_average_removed, l_s_max):
    k_s_max = (4 * c_average_removed) / (1 + l_s_max) - 1
    k_s = int(random.uniform(1, k_s_max + 1))  
    
    return k_s

# test ok

In [58]:
def string_to_remove_length(l_s_max, truck_tour):
    l_t_max = min(len(truck_tour), l_s_max)
    l_t = int(random.uniform(1, l_t_max + 1))
    return l_t

# test ok

In [160]:
def sort_absent_customers(instance, solution, w1, w2, w3, w4, w5):
    
    absent_customers = solution[1]

    def random_sort():
        random.shuffle(absent_customers)
        return absent_customers
        
    def near_dp():
        absent_customers = sorted(absent_customers, key = lambda customer : instance.distances[0][customer])
        return absent_customers
        
    def far_dp():
        absent_customers = sorted(absent_customers, key = lambda customer : (1/instance.distances[0][customer]))
        return absent_customers
        
    def near_tr():
        truck_customers = get_truck_customers(solution[0][0])
        absent_customers = sorted(absent_customers, key = lambda customer : min(instance.distances[t_customer][customer] for t_customer in truck_customers))
        return absent_customers
        
        
    def far_tr():
        truck_customers = get_truck_customers(solution[0][0])
        absent_customers = sorted(absent_customers, key = lambda customer : 1/(min(instance.distances[t_customer][customer] for t_customer in truck_customers)))
        return absent_customers

    # Lista dei blocchi e dei relativi pesi
    sorting_methods = [random_sort, near_dp, far_dp, near_tr, far_tr]
    weights = [w1, w2, w3, w4, w5]

    selected_method = random.choices(sorting_methods, weights, k=1)[0]

    return selected_method()


In [161]:
def truck_tour_time(truck_travel_times, tour):
    total_time = truck_travel_times[0][tour[0]]
    
    for i in range(len(tour) - 1):
        total_time += truck_travel_times[tour[i]][tour[i + 1]]
           
    total_time += truck_travel_times[tour[-1]][0]
    
    return total_time


In [166]:
def drone_tour_time(drone_travel_times, tour):
    total_time = 0
    
    for i in range(len(tour)):
        total_time += (drone_travel_times[i]) * 2
    
    return total_time

In [ ]:
def feasible_truck_tour_positions_calculation(instance, truck_tours, customer):
    feasible_positions = []
    for i in range(len(truck_tours)):
        for j in range (len(truck_tours[i]) + 1): # lo posso aggiungere anche come ultimo
            new_tour = truck_tours[i].copy()
            new_tour.insert(j, customer)
            if (truck_tour_time(instance.t_t, new_tour) < instance.T_t):
                feasible_positions.append[0, i, j]

    return feasible_positions


In [7]:
def is_truck_tour_feasible(instance, tour):
    return (truck_tour_time(instance.t_t, tour) <= instance.T_t)

In [37]:
def is_drone_tour_feasible(instance, tour):
    return (drone_tour_time(instance.t_d, tour) <= instance.T_d)

In [ ]:
def is_drone_eligible(instance, drone_tours, customer):
    if (len(drone_tours) < instance.D):
        return True
    for tour in drone_tours:
        new_tour = tour.copy()
        new_tour.append(customer)
        if (drone_tour_time(instance.t_d, new_tour) <= instance.T_d):
            return True
    
    return False


In [2]:
def insert_customer(customer, pos, solution):
    solution[0][pos[0]][pos[1]].insert(pos[2], customer)
    return solution

In [1]:
def largest_spatial_slack_drone(instance, drone_tours):
    chosen_drone = min(range(len(drone_tours)), key=lambda i: drone_tour_time(instance.t_d, drone_tours[i]))
    return chosen_drone

In [8]:
def cost(instance, solution):
    total_cost = 0
    for truck_tour in solution[0][0]:
        total_cost += truck_tour_time(instance.t_t, truck_tour) * instance.t_speed * instance.C_T

    for drone_tour in solution[0][1]:
        total_cost += drone_tour_time(instance.t_d, drone_tour) * instance.d_speed * instance.C_D

    return total_cost

In [5]:
def select_nearest_neighbors(n_nearest, distances_from_customer):
    indexed_array = [(value, idx) for idx, value in enumerate(distances_from_customer) if idx != 0]
    
    indexed_array.sort(key=lambda x: x[0])
    

    nearest_neighbors = [idx for value, idx in indexed_array[1:n_nearest + 1]] #the depot and the customer itself are escluded
    
    return nearest_neighbors

#test ok

In [12]:
def find_customer_in_vehicle_tours(matrix, target):
    for i, row in enumerate(matrix):
        for j, element in enumerate(row):
            if element == target:
                return [i, j]
    return None

In [13]:
def _2_opt_x(instance, solution, customer, neighbor, customer_tour_index):
    neighbor_index = find_customer_in_vehicle_tours(solution[0][0], neighbor)
    customer_index = [customer_tour_index, next((index for index, c in enumerate(solution[0][0][customer_tour_index]) if c == customer), -1)]
    new_tour_1 = [solution[0][0][neighbor_index[0]][:neighbor_index[1]]] + [solution[0][0][customer_index[0]][customer_index[1]:]]
    new_tour_2 = [solution[0][0][customer_index[0]][:customer_index[1]]] + [solution[0][0][neighbor_index[0]][neighbor_index[1]:]]
    if (is_truck_tour_feasible(instance, new_tour_1) and is_truck_tour_feasible(instance,new_tour_2)):
        solution[0][0][neighbor_index[0]] = new_tour_1
        solution[0][0][customer_index[0]] = new_tour_2

    return solution

In [10]:
def relocate(instance, solution, customer, neighbor, tour_index):
    #customer_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == customer), -1)
    neighbor_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == neighbor), -1)

    proposed_tour = solution[0][0][tour_index].copy()
    proposed_tour.remove(customer)
    proposed_tour.insert(neighbor_index, customer)

    other_tour = solution[0][0][tour_index].copy()
    other_tour.remove(customer)
    other_tour.insert(neighbor_index + 1, customer)

    if (truck_tour_time(instance.t_t, proposed_tour) > truck_tour_time(instance.t_t, other_tour)):
        proposed_tour = other_tour

    solution[0][0][tour_index] = proposed_tour
    return solution




In [11]:
def swap(solution, customer, neighbor, tour_index):
    customer_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == customer), -1)
    neighbor_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == neighbor), -1)

    solution[0][0][tour_index][customer_index] = neighbor
    solution[0][0][tour_index][neighbor_index] = customer

    return solution

In [34]:
def _2_opt(solution, customer, neighbor, tour_index):
    customer_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == customer), -1)
    neighbor_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == neighbor), -1)

    i = min(customer_index, neighbor_index)
    j = max(customer_index, neighbor_index)

    print(i)
    print(j)

    if (i == 0):
        solution[0][0][tour_index][:j+1] = solution[0][0][tour_index][j::-1]
        return solution

    

    solution[0][0][tour_index][i:j+1] = solution[0][0][tour_index][j:i-1:-1]

    return solution

#test ok

In [38]:
def swap_x(instance, solution, customer, neighbor, customer_tour_index):
    customer_index = next((index for index, c in enumerate(solution[0][0][customer_tour_index]) if c == customer), -1)
    neighbor_index = find_customer_in_vehicle_tours(solution[0][1], neighbor)

    modified_truck_tour = solution[0][0][customer_tour_index]
    modified_truck_tour[customer_index] = neighbor

    modified_drone_tour = solution[0][1][neighbor_index[0]]
    modified_drone_tour[neighbor_index[1]] = customer

    if(is_truck_tour_feasible(instance, modified_truck_tour) and is_drone_tour_feasible(instance, modified_drone_tour)):
        solution[0][0][customer_tour_index] = modified_truck_tour
        solution[0][1][neighbor_index[0]] = modified_drone_tour

    return solution

    

metodi "generali" (sotto)

In [27]:
def sweep_removal_operator(instance, solution, sigma):
    c_drones = get_drone_customers(solution[0][1])
    nbSweept = random.randint(0,int(len(c_drones) * sigma))

    grad_seed = random.randint(0, 360)
    c_drones = sort_by_angular(instance.coordinates, c_drones, grad_seed)
    for c in c_drones:
        if (len(solution[1]) < nbSweept):

            #inlcude tour_index calculation inside "remove_drone_customer, params: solution[0][1], c"
            tour_index = next((index for index, drone_tour in enumerate(solution[0][1]) if c in drone_tour), -1)
            solution[0][1][tour_index], solution[1] = remove_drone_customer(solution[0][1][tour_index], solution[1], c)
                               

    return solution

In [28]:
def random_drone_customer_removal(solution, sigma):
    c_drones = get_drone_customers(solution[0])
    q = random.randint(0,int(len(c_drones) * sigma))
    
    
    while (len(solution[1]) < q):
        customer_to_remove = random.choice(c_drones)

        c_drones.remove(customer_to_remove)
        tour_index = next((index for index, drone_tour in enumerate(solution[0][1]) if customer_to_remove in drone_tour), -1)
        solution[0][1][tour_index], solution[1] = remove_drone_customer(solution[0][1][tour_index], solution[0], customer_to_remove)

    return solution

In [29]:
def string_removal(instance, solution, c_average_removed, L_max): # n is the number of customers
    l_s_max = max_string_length(solution[0], instance.N, L_max)
    k_s = n_strings_to_remove(c_average_removed, l_s_max)
    
    c_seed = random.randint(1, instance.n)

    R = []

    c_adj = sort_by_euclidean_distance(instance.distances, c_seed)

    for c in c_adj:
        if len(R) < k_s:
            if any(c in truck_tour for truck_tour in solution[0][0]):
                if c not in solution[1]:
                    tour_index = next((index for index, truck_tour in enumerate(solution[0][0]) if c in truck_tour), -1)
                    if tour_index not in R:
                        l = string_to_remove_length(l_s_max, solution[0][0][tour_index])
                        
                        solution[0][0][tour_index], solution[1] = remove_string(solution[0][0][tour_index], l, c)
                        R.append(tour_index)
            elif any(c in drone_tour for drone_tour in solution[0][1]):
                if c not in solution[1]:
                    tour_index = next((index for index, drone_tour in enumerate(solution[0][1]) if c in drone_tour), -1)
                    solution[0][1][tour_index], solution[1] = remove_drone_customer(solution[0][1][tour_index], c)


    return solution

In [165]:
def recreate(instance, solution, w1, w2, w3, w4, w5, gamma):
    solution[1] = sort_absent_customers(instance, solution, w1, w2, w3, w4, w5)
    for c in solution[1]:
        pos_best = None
        feasible_truck_tour_positions = feasible_truck_tour_positions_calculation(instance, solution[0][0], c)
        for pos in feasible_truck_tour_positions: 
            if pos_best == None or (pos_best != None and cost(instance, insert_customer(c, pos, solution)) < cost(instance, insert_customer(c, pos_best, solution)) and random.random() > (1 - gamma)):
                pos_best = pos
        
        if is_drone_eligible(instance, solution[0][1], c):
            chosen_drone = largest_spatial_slack_drone(instance, solution[0][1])
            if pos_best == None or (pos_best != None and cost(instance, insert_customer(c, [1, chosen_drone, 0], solution)) < cost(instance, insert_customer(c, pos_best, solution)) and random.random() > (1 - gamma)):
                pos_best = [1, chosen_drone, 0]

        if pos_best == None:
            solution[0][0].append([])
            pos_best[0, len(solution[0][0]) - 1, 0]
        
        solution = insert_customer(c, pos_best, solution)
        solution[1].remove(c)

    return solution

In [32]:
def local_search(instance, solution, n_nearest):
    interrupt_truck_tours = False
    interrupt_truck_customers = False

    interrupt_drone_tours = False
    #interrupt_drone_customers = False


    for i in range(len(solution[0][0])):
        if interrupt_truck_tours:
            break
        for customer in range(len(solution[0][0][i])):
            if interrupt_truck_customers:
                interrupt_truck_tours = True
                break
            neighbors = select_nearest_neighbors(n_nearest, instance.distances[customer])
            for neighbor in neighbors:
                if any(neighbor in truck_tour for truck_tour in solution[0][0]):
                    neighbor_index = next((j for j, node in enumerate(solution[0][0][i]) if node == neighbor), None)
                    if neighbor_index == None:
                        new_solution = _2_opt_x(instance, solution.copy(), customer, neighbor, i)
                        if cost(instance, new_solution) < cost(instance, solution):
                            solution = local_search(instance, new_solution, n_nearest)
                            interrupt_truck_customers = True
                            break
                    else: 
                        new_solution = relocate(instance, solution.copy(), customer, neighbor, i)
                        if cost(instance, new_solution) < cost(instance, solution):
                            solution = local_search(instance, new_solution, n_nearest)
                            interrupt_truck_customers = True
                            break

                        new_solution = swap(solution.copy(), customer, neighbor, i)
                        if cost(instance, new_solution) < cost(instance, solution):
                            solution = local_search(instance, new_solution, n_nearest)
                            interrupt_truck_customers = True
                            break

                        new_solution = _2_opt(solution.copy(), customer, neighbor, i)
                        if cost(instance, new_solution) < cost(instance, solution):
                            solution = local_search(instance, new_solution, n_nearest)
                            interrupt_truck_customers = True
                            break

                else:
                    new_solution = swap_x(instance, solution, customer, neighbor, i)
                    if cost(instance, new_solution) < cost(instance, solution):
                        solution = local_search(instance, new_solution, n_nearest)
                        interrupt_truck_customers = True
                        break

            new_solution = shift_t(solution, customer)
            if cost(instance, new_solution) < cost(instance, solution):
                solution = local_search(instance, new_solution, n_nearest)
                interrupt_truck_tours = True
                break

    for i in range(len(solution[0][1])):
        if interrupt_drone_tours:
            break
        for customer in range(len(solution[0][1][i])):
            new_solution = shift_d(solution, customer)
            if cost(instance, new_solution) < cost(instance, solution):
                solution = local_search(instance, new_solution, n_nearest)
                interrupt_drone_tours = True
                break

In [31]:
def initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest):
    A = [c for c in range (1, instance.N)]
    solution = recreate(instance, [[],A], w1, w2, w3, w4, w5, gamma)
    solution = local_search(instance, solution, n_nearest)
    return solution

In [33]:
def perturbation(instance, solution, p_min, p_max, max_unfeasible_swaps):
    p = random.randint(p_min, p_max)
    swaps_executed = 0
    unfeasible_swaps = 0
    while (swaps_executed < p and unfeasible_swaps < max_unfeasible_swaps):
        c1 = random.randint(1, instance.n)
        c2 = random.randint(1, instance.n)
        pos1 = get_position(c1, solution[0])
        pos2 = get_position(c2, solution[0])
        if (is_swap_possible(instance, solution[0], c1, pos2) and is_swap_possible(instance, solution[0], c2, pos1)):
            solution[0][pos1[0]][pos1[1]][pos1[2]] = c2
            solution[0][pos2[0]][pos2[1]][pos2[2]] = c1
            swaps_executed += 1
        else:
            unfeasible_swaps += 1


In [10]:
def SISSRs(instance, w1, w2, w3, w4, w5, gamma, delta, epsilon, iter_max):
    s_0 = initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    while (terminal_condition_met() == False):
        s = ruin_and_recreate(instance, s_curr)
        if cost(instance, s) < cost(instance, s_curr)*(1+delta):
            s_curr = local_search(instance, s)
            if cost(instance, s_curr) < cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_max:
            s_curr = perturbate(instance, s_curr)
            iterations_without_improvement = 0
        delta = delta * epsilon

    return s_best 

           